# 讀出所有方志階層檔，並存於 Book obj 中

In [1]:
filepath = '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC'

In [2]:
import os

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(filepath):
    for file in f:
        if '._' not in file and '.xml' in file and file != 'bookId.xml':
            files.append(os.path.join(r, file))

print(len(files))
print(files[0:5])

3998
['/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/35392.xml', '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/97486.xml', '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/35386.xml', '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/92850.xml', '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/36842.xml']


In [3]:
import xmltodict, json

Books = {}

def singleChapter(INchapter, INparent, chapterTitles):
    _title = INchapter['title']
    
    _from = ""
    if '@from' in INchapter:
        _from = INchapter['@from']
    
    _to = ""
    if '@to' in INchapter:
        _to = INchapter['@to']
        
    _level = ""
    if '@level' in INchapter:
        _level = INchapter['@level']
        
    _n = ""
    if '@n' in INchapter:
        _n = INchapter['@n']

    # 'parent' = [sub]
#     print(INparent, _title)
    if INparent not in chapterTitles:
        chapterTitles[INparent] = [_title]
    elif _title not in chapterTitles[INparent]:
        chapterTitles[INparent].append(_title)
        
    # [sub] = [parent]
#     if _title not in chapterTitles:
#         chapterTitles[_title] = [INparent]
#     elif INparent not in chapterTitles[_title]:
#         chapterTitles[_title].append(INparent)
    
def section(INarray, INtitle, chapterTitles):
#     print("section::", INtitle, type(INarray), INarray)
    if type(INarray) == dict:
        singleChapter(INarray, INtitle, chapterTitles)
        if 'section' in INarray:
            section(INarray['section'], INarray['title'], chapterTitles) 
        
    elif type(INarray) == list:    
        for ch in INarray:
            section(ch, INtitle, chapterTitles)
            if 'section' in ch:
                section(ch['section'], ch['title'], chapterTitles)
                
    else:
        print(INarray)
        print("ERROR:", type(INarray))
        
    return chapterTitles

# file: [35393].xml
def recursiveSection(INdata):
    chapterTitles = {}
    book = INdata['title']

    if 'section' in INdata:
        titleObj = section(INdata['section'], book, chapterTitles)
    return titleObj

def parseFile(file):
    with open(file) as fp:
        xmlStr = fp.read()
        convertedDict = xmltodict.parse(xmlStr);
        jsonStr = json.dumps(convertedDict, ensure_ascii=False)
        jsonObj = json.loads(jsonStr)

#         print(jsonObj)
        book = jsonObj['book']

        titleObj = recursiveSection(book)
#         print(titleObj)
    
        rstart = file.rfind('/')
        rend = file.rfind('.')
        # file: [35393].xml
        bid = file[rstart+1:rend]
        
        bname = list(titleObj.keys())[0]         
        Books[bid] = {'book': bname, 'data': titleObj}
    
# print(files[0])
# parseFile(files[0])
# print(Books)
# parseFile("/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志/TOC/97725.xml")

In [4]:
# root = AnyNode(id="root")
for file in files:
    parseFile(file)
print(len(Books))
# parseFile(files[0])

3998


In [5]:
# the first 
# print(Books[list(Books.keys())[0]])
# print(Books['17106'])
# print(Books['97725'])
print(Books['35282'])


{'book': '涇縣續志', 'data': {'涇縣續志': ['山澤附（闕）', '節婦（闕）', '涇縣續志序', '涇縣續志纂修姓氏', '涇縣續志目錄', '卷一', '卷二', '卷三', '卷四', '卷五', '卷六', '卷七', '卷八', '卷九'], '卷一': ['學校', '書院', '壇廟', '公館', '橋梁', '津渡', '古蹟', '寺觀'], '書院': ['學田'], '古蹟': ['冡墓'], '卷二': ['食貨', '職官表', '選舉表'], '食貨': ['蠲賑', '恩賚', '户口', '積貯'], '選舉表': ['例仕', '吏仕', '武選舉', '武職', '贈封', '歷襲'], '卷三': ['人物'], '人物': ['名臣', '宦業', '忠節', '孝友', '儒林', '文苑', '武功', '懿行', '尚義', '五世同堂', '百歳', '隱逸', '藝術', '寓賢'], '卷四': ['人物'], '卷五': ['人物'], '卷六': ['人物'], '卷七': ['列女'], '列女': ['已旌', '烈婦：已旌', '貞女', '壽婦', '節婦：已故待旌', '烈婦', '孝女', '現年節婦'], '卷八': ['列女'], '卷九': ['藝文', '災祥', '附刻煙石嶺禁碑并\uedce字']}}


# 哪幾本方志階層不用修正 

In [6]:
import sys

def showLevelGap(level):
    x = ""
    for idx in range(0, level):
        x += "==="
    return x
    
def showSubChapter(subs, data, level, uppers):
    if level > 10:
        sys.exit()

    for s in subs:
        prefix = showLevelGap(level)
        print(prefix, s)
        if s not in data:
            continue

        if level == 1:
#             print("LEVEL 1::", uppers)
            uppers = []
        else:
            if s in uppers:
                continue
            else:
                uppers.append(s)
            
        nxtSubs = data[s]
        showSubChapter(nxtSubs, data, level+1, uppers)
    
# read one gazetters
def showGazetter(bid):
    if bid not in Books:
        print("ERROR!!", bid)
        return
    gazetteer = Books[bid]
    title = gazetteer['book']
    print(title)

    data = gazetteer['data']
#     print(data)
    
    firstLevels = data[title]
    uppers = []
    showSubChapter(firstLevels, data, 1, uppers)
            
# showGazetter('35392')
# showGazetter('17106')
# showGazetter('97725')
# showGazetter('35282')


In [7]:
import sys
import re

def isChineseOnly(word):
    if type(word) != str:
        return
            
    founds = re.findall(r'[\u4e00-\u9fff]+', word)
    if len(founds) != 1:
        pass
#         print("Something not Chinese:", word, founds)

def checkSubChapter(subs, data, level, uppers, limitLen):
    if level > 10:
        sys.exit()

    for s in subs:        
        # ##FIX##, the s could be dict in 97972. {'hi': '田產附', '#text': '義學'}
        if type(s) != str:
            return False
        
        isChineseOnly(s)
        
        if len(s) > limitLen:
            return False
        
        if s not in data:
            continue
            
        if level == 1:
            uppers = []
        else:
            if s in uppers:
                continue
            else:
                uppers.append(s)

        nxtSubs = data[s]
        if checkSubChapter(nxtSubs, data, level+1, uppers, limitLen) == False:
            return False
    return True

# check gazetter's chapter categories are correct or not
def checkGazetter(bid, limitLen):
    if bid not in Books:
        print("ERROR!!", bid)
        return False
    gazetteer = Books[bid]
    title = gazetteer['book']

    data = gazetteer['data']
    firstLevels = data[title]
    return checkSubChapter(firstLevels, data, 1, [], limitLen)

# checkGazetter('36789', 3)
# checkGazetter('98012', 3)
# checkGazetter('97725', 3)
# checkGazetter('97972', 10)
checkGazetter('97308', 10)


False

In [8]:
gazetters3 = []
for b in Books:
    if checkGazetter(b, 3):
        gazetters3.append(b)
print(len(gazetters3))
print(gazetters3)

17
['98145', '97953', '97622', '94355', '97017', '93658', '93659', '94291', '92903', '37039', '71417', '93555', '98144', '93051', '16649', '93602', '97806']


In [9]:
showGazetter('97806')

南川縣志
=== 沿革
=== 星野
=== 形勝
=== 城池
=== 公署
=== 學校
=== 風俗
=== 戶口
=== 田賦
=== 山川
=== 橋樑
=== 祠觀
=== 古蹟
=== 秩官
=== 節烈
=== 名宦
=== 鄉賢
=== 人物


## Merge the categories

In [10]:
categories = {}

def addCategory(parent, subarr):
    if type(parent) != str:
        return

    if parent not in categories:
        # remove the duplicated
        categories[parent] = list(set(subarr))
    else:
        existList = set(categories[parent])
        subList = set(subarr)
        differList = subList - existList
        categories[parent].extend(list(differList))
        
def mergeSubChapter(subs, data, level, uppers):
    if level > 10:
        sys.exit()

    for s in subs:
        if s not in data:
            if level == 1:
                addCategory(s, [])
            continue

        if level == 1:
#             print("LEVEL 1::", uppers)
            uppers = []
        else:
            if s in uppers:
                continue
            else:
                uppers.append(s)

        nxtSubs = data[s]
        addCategory(s, nxtSubs)
        mergeSubChapter(nxtSubs, data, level+1, uppers)
    
# read one gazetters
def mergeGazetter(bid):
    if bid not in Books:
        print("ERROR!!", bid)
        return
    gazetteer = Books[bid]
    title = gazetteer['book']
    print(title)

    data = gazetteer['data']
#     print(data)
    
    firstLevels = data[title]
    uppers = []
    mergeSubChapter(firstLevels, data, 1, uppers)

In [11]:
for bid in gazetters3:
    mergeGazetter(bid)
# showGazetter('97622')
# mergeGazetter('97622')
categories

黟縣續志
潛江縣志稿
興濟縣志
梅菉賦志
徽縣志
南江縣備造新編志書清册
冕寧縣志清册
續定海縣志
昭覺縣志稿
荔波縣志稿
隨志
打拉池縣丞志
灤志補
麗江縣志
翁源縣志
合水縣志
南川縣志


{'人物志': ['列女', '尚義', '仕蹟', '科第', '聖蹟', '貞烈', '孝行', '宦蹟', '壽官', '封贈', '國戚'],
 '列女': ['節孝', '節烈', '貞女'],
 '建置': [],
 '政績錄': ['訓導', '教諭', '典吏', '縣丞', '城守'],
 '兵事錄': [],
 '耆舊錄': ['人類', '名宦祠', '戶口', '忠孝祠', '鄉賢祠', '節烈祠', '氏族'],
 '宗教': [],
 '實業': [],
 '卷七': ['山', '市鎮', '古蹟', '地理', '道路', '物產', '坊表', '學堂', '水', '祠廟', '橋梁'],
 '圖例': [],
 '天文志': ['祥異', '分野'],
 '地理志': ['沿革', '疆域'],
 '山川志': [],
 '建置志': ['古蹟',
  '徭役',
  '鋪舍',
  '窑厰',
  '坊表',
  '戶口',
  '城池',
  '官地',
  '支應官',
  '祠廟',
  '塩政',
  '莊村',
  '公署',
  '職官',
  '鎮店',
  '馬政',
  '屯所',
  '田賦',
  '寺觀'],
 '風土志': ['風土', '鄉儀', '時序', '土產'],
 '學校志': ['儒學', '社學', '文廟', '學額', '義學'],
 '金石志': [],
 '證': [],
 '序言': [],
 '自叙': [],
 '首': [],
 '東': ['武今', '职官古', '武古', '舉貢古', '封典古', '功名今'],
 '西': [],
 '北': [],
 '中': [],
 '風俗': [],
 '嵗例': [],
 '善舉': [],
 '災祥': ['疫', '兵事'],
 '诗': [],
 '徽縣志': ['學正',
  '寺院',
  '進士',
  '古蹟',
  '訓導',
  '兵防',
  '徽縣志',
  '沿革',
  '里所',
  '疆域',
  '仙釋',
  '知州',
  '税課',
  '典史',
  '祥異',
  '教諭',
  '孝友',
  '城池',
  '貢賦',
  '舉人',
  '風俗',
  '山川',
  

In [12]:
removeList = ['序', '下卷', '上卷', '卷七', '序言', '自叙', '徽縣志']

# remove the elements which should not be the class
def removeElements(ctgrs, removes):
    newCategory = {}
    for key in ctgrs.keys():
        subArr = ctgrs[key]
        if key in removes:
            # remove the class and move the sub classes to upper layer
            for cs in subArr:
                if cs not in removeList and cs not in newCategory:
                    newCategory[cs] = []
        else:
            newCategory[key] = subArr
            
    return newCategory


categs = removeElements(categories, removeList)
categs

{'人物志': ['列女', '尚義', '仕蹟', '科第', '聖蹟', '貞烈', '孝行', '宦蹟', '壽官', '封贈', '國戚'],
 '列女': ['節孝', '節烈', '貞女'],
 '建置': [],
 '政績錄': ['訓導', '教諭', '典吏', '縣丞', '城守'],
 '兵事錄': [],
 '耆舊錄': ['人類', '名宦祠', '戶口', '忠孝祠', '鄉賢祠', '節烈祠', '氏族'],
 '宗教': [],
 '實業': [],
 '山': [],
 '市鎮': [],
 '古蹟': [],
 '地理': [],
 '道路': [],
 '物產': [],
 '坊表': [],
 '學堂': [],
 '水': [],
 '祠廟': [],
 '橋梁': [],
 '圖例': [],
 '天文志': ['祥異', '分野'],
 '地理志': ['沿革', '疆域'],
 '山川志': [],
 '建置志': ['古蹟',
  '徭役',
  '鋪舍',
  '窑厰',
  '坊表',
  '戶口',
  '城池',
  '官地',
  '支應官',
  '祠廟',
  '塩政',
  '莊村',
  '公署',
  '職官',
  '鎮店',
  '馬政',
  '屯所',
  '田賦',
  '寺觀'],
 '風土志': ['風土', '鄉儀', '時序', '土產'],
 '學校志': ['儒學', '社學', '文廟', '學額', '義學'],
 '金石志': [],
 '證': [],
 '首': [],
 '東': ['武今', '职官古', '武古', '舉貢古', '封典古', '功名今'],
 '西': [],
 '北': [],
 '中': [],
 '風俗': [],
 '嵗例': [],
 '善舉': [],
 '災祥': ['疫', '兵事'],
 '诗': [],
 '學正': [],
 '寺院': [],
 '進士': [],
 '訓導': [],
 '兵防': [],
 '沿革': [],
 '里所': [],
 '疆域': [],
 '仙釋': [],
 '知州': [],
 '税課': [],
 '典史': [],
 '祥異': [],
 '教諭': [],
 '孝友': [

# Check length > 4

In [13]:
def checkChineseOnly(INctgrs):
    for key in INctgrs.keys():
        isChineseOnly(key)
        subArr = INctgrs[key]
        
        for elm in subArr:
            isChineseOnly(elm)
            
checkChineseOnly(categs)

In [14]:
gazetters4 = []
for b in Books:
    if b not in gazetters3 and checkGazetter(b, 4):
        gazetters4.append(b)
print(len(gazetters4))
print(gazetters4)

41
['97679', '98012', '93915', '97523', '97107', '97813', '97346', '93749', '92932', '93252', '36985', '35748', '92482', '98068', '40997', '71730', '93761', '36561', '35454', '17180', '16646', '36574', '94404', '16640', '35598', '94205', '93889', '845', '346', '36416', '16696', '852', '380', '17313', '97805', '17035', '643', '836', '97648', '97810', '35792']


In [15]:
# showGazetter('97679')
for bid in gazetters4:
    mergeGazetter(bid)
# showGazetter('97622')
# mergeGazetter('97622')
categories

玉山縣志
昭萍志略
鄰水縣續志
餘杭縣志稿
朝城縣志略
巴州志略
三續淄川縣志
朝邑縣志
鄖陽志補
富民縣志
普安縣志
敦煌縣志
臺拱縣文獻紀要
巫山縣志
齊乘
濱州志
西林縣志
雙山縣志
宜川縣志
商河縣志
仁化縣志
梨樹縣志
靖海衛志
臨朐縣志
趙城縣志
鳳縣志
懋功縣志
連山綏瑤廳志
始興縣志
重修儀徵縣志
壽州志
南澳志
百色廳志
豐乘
開縣志
古海陵縣志
玉門縣志
續番禺縣志
略陽縣志
巫山縣志
長陽縣志


{'人物志': ['列女',
  '尚義',
  '仕蹟',
  '科第',
  '聖蹟',
  '貞烈',
  '孝行',
  '宦蹟',
  '壽官',
  '封贈',
  '國戚',
  '列女傳',
  '列傳',
  '流寓',
  '先賢',
  '孝義',
  '文學',
  '武勳',
  '旌異',
  '進士',
  '例貢',
  '薦舉',
  '鄉賢',
  '歲貢',
  '舉人',
  '名賢',
  '貞節',
  '儒行',
  '孝友',
  '義行',
  '宦績下',
  '隱逸',
  '宦績中',
  '忠烈',
  '方外',
  '僑寓',
  '宦績上',
  '藝術'],
 '列女': ['節孝', '節烈', '貞女', '賢惠', '貞烈'],
 '建置': ['古蹟',
  '津梁',
  '收卹',
  '行署',
  '坊表',
  '邱墓',
  '城池',
  '倉庫',
  '驛舗',
  '公署',
  '壇廟',
  '陂塘'],
 '政績錄': ['訓導', '教諭', '典吏', '縣丞', '城守'],
 '兵事錄': [],
 '耆舊錄': ['人類', '名宦祠', '戶口', '忠孝祠', '鄉賢祠', '節烈祠', '氏族'],
 '宗教': [],
 '實業': [],
 '卷七': ['山',
  '市鎮',
  '古蹟',
  '地理',
  '道路',
  '物產',
  '坊表',
  '學堂',
  '水',
  '祠廟',
  '橋梁',
  '秩官',
  '武備志',
  '人物志',
  '經政畧',
  '輿地志'],
 '圖例': [],
 '天文志': ['祥異', '分野'],
 '地理志': ['沿革', '疆域'],
 '山川志': ['古蹟', '圍井', '丘墓附'],
 '建置志': ['古蹟',
  '徭役',
  '鋪舍',
  '窑厰',
  '坊表',
  '戶口',
  '城池',
  '官地',
  '支應官',
  '祠廟',
  '塩政',
  '莊村',
  '公署',
  '職官',
  '鎮店',
  '馬政',
  '屯所',
  '田賦',
  '寺觀',
  '文廟',
  '城防',
  '驛舖',
  '書院',

In [16]:
gazettersN = []
for b in Books:
#     print(b)
    if checkGazetter(b, 10):
        gazettersN.append(b)
print(len(gazettersN))

3097


In [17]:
specialWords = ['卷', '一', '二', '三', '四', '五', '六', '七', '八', '九', '十', ]

# test

In [18]:
# dupbooks = {}
# for key in bookTitles:
# #     print(bookTitles[key])
#     if len(bookTitles[key]) > 1:
# #         print(key, bookTitles[key])
#         dupbooks[key] = bookTitles[key]
# print(len(dupbooks))

In [19]:
# import pandas as pd

# resDf = pd.DataFrame.from_dict(dupbooks, orient='index')
# # print(resDf)
# resDf.sort_index(inplace=True)
# resDf.head()

# print(len(resDf))

In [20]:
# outputPath = '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志'
# write_file_to = '{0}/{1}'.format(outputPath, 'gazetteers_20190811_v2.xlsx')
# resDf.to_excel(write_file_to, index=True)

In [21]:
# import pandas as pd

# resDf = pd.DataFrame(chapterTitles, columns=['section'])
# resDf.sort_values(by=['section'], inplace=True)
# resDf.head()

# print(len(resDf))

In [22]:
# outputPath = '/Volumes/backup_128G/z_repository/PTC_data/DADH_2019/0_方志'
# write_file_to = '{0}/{1}'.format(outputPath, 'gazetteers_20190811_v1.xlsx')
# resDf.to_excel(write_file_to, index=False)

In [23]:
# count = 0
# for key in chapterTitles:
#     print(key, chapterTitles[key])
#     count += 1
#     if count > 5:
#         break

In [24]:
# # Remove bookname
# booknamecnt = 0
# for sec in chapterTitles:
# #     print(type(sec))
#     if sec == None:
#         continue
#     titles = chapterTitles[sec]
    
#     for tit in titles:
#         if tit == None:
#             continue
# #         print(sec)
#         if sec.startswith(tit):
#             if tit not in bookTitles:
#                 # remove the same section names in parent and sub section both
#                 if sec == tit:
#                     chapterTitles[sec].remove(tit)
#                     print(sec, tit)
#             else:
# #                 print(tit)
#                 chapterTitles[sec].remove(tit)

#             booknamecnt += 1
        
#         # remove parent section is book
# #         if tit in bookTitles:
# #             chapterTitles[sec].remove(tit)
# #             booknamecnt += 1
# #             print(sec, tit)
            
# print(booknamecnt)
# # print(len(chapterTitles))

In [25]:
# bookTitles['成安縣志']

In [26]:
# remove parent section is book
# for sec in chapterTitles:
# #     print(type(sec))
#     if sec == None:
#         continue
#     titles = chapterTitles[sec]
    
#     for tit in titles:
#         if tit == None:
#             continue

#         if tit in bookTitles:
#             # remove the parent section name which is the same as bookname
#             chapterTitles[sec].remove(tit)
            
# print(len(chapterTitles))

In [27]:
# 僊居水卷之一
# type(chapterTitles)

In [28]:
# import re

# for key in chapterTitles:
#     titles = chapterTitles[key]
#     for t in titles:
#         x = re.search(".*卷[首之上下一二三四五六七八九十]*", t)
#         if x != None:
#             print(t, x)
#             chapterTitles[key].remove(t)

In [29]:
# chapterTitles